<a href="https://colab.research.google.com/github/01PrathamS/BERT_from_scratch/blob/main/notebooks/FineTuning_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, Dataset

raw_datasets = load_dataset("quac")

README.md:   0%|          | 0.00/17.2k [00:00<?, ?B/s]

quac.py:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

The repository for quac contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/quac.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/11567 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['dialogue_id', 'wikipedia_page_title', 'background', 'section_title', 'context', 'turn_ids', 'questions', 'followups', 'yesnos', 'answers', 'orig_answers'],
        num_rows: 11567
    })
    validation: Dataset({
        features: ['dialogue_id', 'wikipedia_page_title', 'background', 'section_title', 'context', 'turn_ids', 'questions', 'followups', 'yesnos', 'answers', 'orig_answers'],
        num_rows: 1000
    })
})

In [8]:
print("id: ", raw_datasets["train"][0]["dialogue_id"][len(raw_datasets["train"][0]["dialogue_id"])-1:])
print("***")
print("Context: ", raw_datasets["train"][0]["context"])
print("***")
print("Question: ", raw_datasets["train"][0]["questions"])
print("***")
print("Answer: ", raw_datasets["train"][0]["answers"]['texts'])

id:  1
***
Context:  According to the Indian census of 2001, there were 30,803,747 speakers of Malayalam in Kerala, making up 93.2% of the total number of Malayalam speakers in India, and 96.7% of the total population of the state. There were a further 701,673 (2.1% of the total number) in Karnataka, 557,705 (1.7%) in Tamil Nadu and 406,358 (1.2%) in Maharashtra. The number of Malayalam speakers in Lakshadweep is 51,100, which is only 0.15% of the total number, but is as much as about 84% of the population of Lakshadweep. In all, Malayalis made up 3.22% of the total Indian population in 2001. Of the total 33,066,392 Malayalam speakers in India in 2001, 33,015,420 spoke the standard dialects, 19,643 spoke the Yerava dialect and 31,329 spoke non-standard regional variations like Eranadan. As per the 1991 census data, 28.85% of all Malayalam speakers in India spoke a second language and 19.64% of the total knew three or more languages.  Large numbers of Malayalis have settled in Bangalore

In [11]:
# Loop through each example in the "train" split

question_train = []
answer_train = []
answer_starts_t = []
context_train = []

for example in raw_datasets["train"]:
  context_train.append(example["context"])
  answer_train.append(example["answers"]["texts"][0])
  answer_starts_t.append(example["answers"]["answer_starts"][0])
  question_train.append(example["questions"][0])

# Loop through each example in validation split

question_val = []
answer_val = []
answer_starts_v = []
context_val = []

for example in raw_datasets["validation"]:
  context_val.append(example["context"])
  answer_val.append(example['answers']['texts'])
  answer_starts_v.append(example["answers"]["answer_starts"][0])
  question_val.append(example["questions"][0])

len(answer_train), len(answer_val)

(11567, 1000)

In [12]:
id_train = []
id_val = []

for i in range(len(raw_datasets["train"])):
  id_train.append(i)
for i in range(len(raw_datasets["validation"])):
  id_val.append(i)

In [13]:
answer_t = Dataset.from_dict({"texts": answer_train, "answer_starts": answer_starts_t})
answer_v = Dataset.from_dict({"texts": answer_val, "answer_starts": answer_starts_v})

train_data = {"question": question_train, "answer": answer_t, "context": context_train, "id": id_train}
val_data = {"question": question_val, "answer": answer_v, "context": context_val, "id": id_val}

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

data = {
    "train": train_dataset,
    "val": val_dataset
}

data

{'train': Dataset({
     features: ['question', 'answer', 'context', 'id'],
     num_rows: 11567
 }),
 'val': Dataset({
     features: ['question', 'answer', 'context', 'id'],
     num_rows: 1000
 })}

In [14]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
context = data["train"][0]["context"]
question = data["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] Where is Malayali located? [SEP] According to the Indian census of 2001, there were 30, 803, 747 speakers of Malayalam in Kerala, making up 93. 2 % of the total number of Malayalam speakers in India, and 96. 7 % of the total population of the state. There were a further 701, 673 ( 2. 1 % of the total number ) in Karnataka, 557, 705 ( 1. 7 % ) in Tamil Nadu and 406, 358 ( 1. 2 % ) in Maharashtra. The number of Malayalam speakers in Lakshadweep is 51, 100, which is only 0. 15 % of the total number, but is as much as about 84 % of the population of Lakshadweep. In all, Malayalis made up 3. 22 % of the total Indian population in 2001. Of the total 33, 066, 392 Malayalam speakers in India in 2001, 33, 015, 420 spoke the standard dialects, 19, 643 spoke the Yerava dialect and 31, 329 spoke non - standard regional variations like Eranadan. As per the 1991 census data, 28. 85 % of all Malayalam speakers in India spoke a second language and 19. 64 % of the total knew three or more langua

In [16]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

[CLS] Where is Malayali located? [SEP] According to the Indian census of 2001, there were 30, 803, 747 speakers of Malayalam in Kerala, making up 93. 2 % of the total number of Malayalam speakers in India, and 96. 7 % of the total population of the state. There were a further 701, 673 ( 2. 1 % of the total number ) in Karnataka, 557, 705 ( 1. 7 % ) in Tamil Nadu and 406 [SEP]
[CLS] Where is Malayali located? [SEP]. 7 % of the total population of the state. There were a further 701, 673 ( 2. 1 % of the total number ) in Karnataka, 557, 705 ( 1. 7 % ) in Tamil Nadu and 406, 358 ( 1. 2 % ) in Maharashtra. The number of Malayalam speakers in Lakshadweep is 51, 100, which is only 0. 15 % of the total number, but [SEP]
[CLS] Where is Malayali located? [SEP] 7 % ) in Tamil Nadu and 406, 358 ( 1. 2 % ) in Maharashtra. The number of Malayalam speakers in Lakshadweep is 51, 100, which is only 0. 15 % of the total number, but is as much as about 84 % of the population of Lakshadweep. In all, Mala

In [18]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [21]:
inputs = tokenizer(
    data["train"][2:6]["question"],
    data["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")


The 4 examples gave 43 features.
Here is where each comes from: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3].


In [24]:
train_data = data["train"]
answers = train_data[2:6]["answer"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_starts"][0]  # Update field name
    end_char = answer["answer_starts"][0] + len(answer["texts"][0])  # Update field name
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions


([25,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  67,
  28,
  0,
  0,
  0,
  0,
  22,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [41,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  30,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  74,
  35,
  0,
  0,
  0,
  0,
  47,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0])

In [26]:
idx= 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["texts"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: 19-year-old Cove Reber was announced as their new permanent lead singer., labels give: 19 - year - old Cove Reber was announced as their new permanent lead singer.


In [29]:
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["texts"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: 19-year-old Cove Reber was announced as their new permanent lead singer., decoded example: [CLS] What do we know about Cove Reber? [SEP] their lives... Saosin is a band on a completely different level. All these dudes are freaks about music. " Reber ' s addition to the band was difficult, for the more experienced Green was the center piece of the band in the eyes of Saosin ' s fans. Many fans consider the time with Green to be something entirely different from the time with Reber. There are still distinct fans of both [SEP]


In [31]:
# Preprocessing Training data

max_length = 384
stride = 128

def preprocess_training_example(examples):
  questions = [q.strip() for q in examples["question"]]
  inputs = tokenizer(
      questions,
      examples["context"],
      max_length=max_length,
      truncation="only_second",
      stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
  )

  offset_mapping = inputs.pop("offset_mapping")
  sample_map = inputs.pop("overflow_to_sample_mapping")
  answers = examples["answer"]
  start_positions = []
  end_positions = []

  for i, offset in enumerate(offset_mapping):
    sample_idx = sample_map[i]
    answer = answers[sample_idx]
    start_char = answer["answer_starts"][0]
    end_char = answer["answer_starts"][0] + len(answer["texts"][0])
    sequence_ids = inputs.sequence_ids(i)

    idx = 0
    while sequence_ids[idx] != 1:
      idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
      idx += 1
    context_end = idx - 1

    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
      start_positions.append(0)
      end_positions.append(0)

    else:
      idx = context_start
      while idx <= context_end and offset[idx][0] <= start_char:
        idx += 1
      start_positions.append(idx - 1)

      idx = context_end
      while idx >= context_start and offset[idx][1] >= end_char:
        idx -= 1
      end_positions.append(idx + 1)

  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions

  return inputs

In [32]:
train_dataset = data["train"].map(preprocess_training_example, batched=True, remove_columns=data["train"].column_names,)
len(data["train"]), len(train_dataset)

Map:   0%|          | 0/11567 [00:00<?, ? examples/s]

(11567, 25362)

In [37]:
# preprocess validation data


def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [38]:
validation_dataset = data["val"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=data["val"].column_names,
)
len(data["val"]), len(validation_dataset)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

(1000, 2452)

In [39]:
small_eval_set = data["val"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=data["val"].column_names,
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [40]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [ ]:
 import torch
 from transformers import AutoModelForQuestionAnswering

 eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
 eval_set_for_model.set_format("torch")

 device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
 batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
 trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(device)

 with torch.no_grad():
  outputs = trained_model(**batch)

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]